从鸭子类型的代表特征动态协议，到使接口更明确、能验证实现是否符合规定的抽象基类（Abstract Base Class, ABC）。

抽象基类的常见用途：实现接口时作为超类使用。

抽象基类与描述符和元类一样，是用于构建框架的工具。

除了抽象基类，每个类都有接口：类实现或继承的公开属性（方法或数据属性），包括特殊方法，如 \_\_getitem__ 或 \_\_add__

按照定义，受保护的属性和私有属性不在接口中。

接口是实现特定角色的方法集合。协议与继承没有关系。一个类可能会实现多个接口，从而让实例扮演多个角色。

协议是接口，但不是正式的（只由文档和约定定义），因此协议不能像正式接口那样施加限制。一个类可能只实现部分接口。

对 Python 程序员来说，X 类对象、X 协议、X 接口都是一个意思。

如果没有 \_\_iter__ 和 \_\_contains__ 方法，Python 会调用 \_\_getitem__ 方法，设法让迭代和 in 运算符可用。

Python 中的迭代是鸭子类型的一种极端形式：为了迭代对象，解释器会尝试调用两个不同的方法。

In [1]:
class Foo:
    def __getitem__(self, pos):
        return range(0, 30, 10)[pos]

f = Foo()
f[1]

In [2]:
for i in f:
    print(i)

0
10
20


In [3]:
20 in f

True

In [4]:
15 in f

False

如果遵守既定协议，很有可能增加利用现有的标准库和第三方代码的可能性，这得益于鸭子类型。

In [5]:
from random import shuffle

l = list(range(10))
shuffle(l)
l

[1, 6, 3, 0, 2, 9, 7, 8, 4, 5]

In [6]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

FrenchDeck 只实现了不可变的序列协议，可变的序列还必须提供 \_\_setitem__ 方法。

In [7]:
deck = FrenchDeck()
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

猴子补丁（monkey patch）：在运行时修改类或模块，而不改动源码。

In [8]:
def set_card(deck, position, card):
    deck._cards[position] = card

FrenchDeck.__setitem__ = set_card

shuffle(deck)
deck[:5]

[Card(rank='3', suit='diamonds'),
 Card(rank='3', suit='hearts'),
 Card(rank='8', suit='clubs'),
 Card(rank='4', suit='spades'),
 Card(rank='6', suit='spades')]

鸭子类型：对象的类型无关紧要，只要实现了特定的协议即可。
- 忽略对象的真正类型，转而关注对象有没有实现所需的方法、签名和语义

白鹅类型：只要 cls 是抽象基类，即 cls 的元类是 abc.ABCMeta ，就可以使用 isinstance(obj, cls)

collections.abc 中有很多有用的抽象类（Python 标准库的 numbers 模块中还有一些）。

Python 的抽象基类还有一个重要的实用优势：可以使用 register 类方法在终端用户的代码中把某个类“声明”为一个抽象基类的“虚拟子类”
- 为此，被注册的类必须满足抽象基类对方法名和签名的要求，最重要的是满足底层的语义契约
- 但是，开发那个类时不用了解抽象基类，更不用继承抽象基类

有时，为了让抽象基类识别子类，甚至不用注册。抽象基类的本质就是几个特殊方法，此外，不要在生产代码中定义抽象基类（或元类）

不要滥用 isinstance 检查，用多了可能导致代码异味，即表明面向对象设计得不好；此时应该使用多态，即采用一定的方式定义类，让解释器把调用分派给正确的方法，而不是用 if/elif/else 块硬编码分派逻辑。

In [9]:
class Struggle:
    def __len__(self):
        return 23

from collections import abc

isinstance(Struggle(), abc.Sized)

True

使用鸭子类型处理单个字串或由字符串组成的可迭代对象

EAFP 风格，即“取得原谅比获得许可容易”

```python
try:  # 假设是单个字符串
    # 逗号替换为空格。拆分为列表
    field_names = field_names.replace(',', ' ').split()
except AttributeError:  # 假设是由字符串组成的可迭代对象
    pass

# 保存副本
field_names = tuple(field_names)
```

导入（加载并编译模块时），Python 不会检查抽象方法的实现，在运行时实例化类时才会真正检查。

在 collections.abc 中，每个抽象基类的具体方法都是作为类的公开接口实现的，因此不用知道实例的内部结构。

collections.abc 的一些基类：
- Iterable、Container、Sized
- Sequence、Mapping、Set
- MappingView
- Callable、Hashable
    - 主要作用是为内置函数 isinstance 提供支持，以一种安全的方式判断对象能不能调用或散列
    - 若想检查是否能调用，可以使用内置的 callable() 函数

In [10]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]
    
    # 支持洗牌
    def __setitem__(self, position, value):
        self._cards[position] = value
    
    # 继承 MutableSequence 类必须实现
    def __delitem__(self, position):
        del self._cards[position]

    # 继承 MutableSequence 类必须实现
    def insert(self, position, value):
        self._cards.insert(position, value)

<ipython-input-10-f221085aea67>:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  class FrenchDeck2(collections.MutableSequence):


numbers 包定义了抽象基类的数字塔，Number 是位于最顶端的超类
- Number
- Complex
- Real
- Rational
- Integral

为了满足检查的需要，你或者你的 API 用户始终可以把兼容的类型注册为 numbers.Integer 的虚拟子类。

如果程序需要 Decimal 的精度，要防止与其他低精度数字类型混淆，尤其是浮点数

抽象方法可以有实现代码。即使实现了，子类也必须覆盖抽象方法，但是在子类中可以使用 super() 函数调用抽象方法，为它添加功能，而不是从头开始实现。

抽象基类可以提供具体方法，只要依赖接口中的其他方法就行。

In [11]:
# 抽象基类

import abc

# 自定义的抽象基类要继承 abc.ABC
class Tombola(abc.ABC):

    # 抽象方法装饰器
    # 定义体通常只有文档字符串
    @abc.abstractmethod
    def load(self, iterable):
        """从可迭代对象中添加元素"""

    # 抽象方法
    @abc.abstractmethod
    def pick(self):  # <3>
        """随机删除元素，然后将其返回

        如果实例为空，这个方法应该抛出 `LookupError`
        """
    
    # 可以包含具体方法
    # 只能依赖抽象基类定义的接口
    # 即只能使用抽象基类中的其他具体方法、抽象方法或特性
    def loaded(self):
        """如果至少有一个元素，返回 `True` ，否则返回 `False` """
        return bool(self.inspect())

    # 具体方法
    def inspect(self):
        """返回一个有序元组，由当前元素构成"""
        items = []
        while True:  # <6>
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)  # <7>
        return tuple(sorted(items))

In [12]:
# 不符合要求的子类
class Fake(Tombola):
    def pick(self):
        return 13

Fake

__main__.Fake

In [13]:
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract methods load

声明抽象基类最简单的方式是继承 abc.ABC 或其他抽象基类。

声明抽象类方法的推荐方式是：
```python
class MyABC(abc,ABC):
    @classmethod
    @abc.abstractmethod
    def an_abstract_classmethod(cls, ...):
        pass
```

在函数上堆叠装饰器的顺序通常很重要
- 与其他方法描述符一起使用时，abstractmethod() 应该放在最里层

In [14]:
import random

# 扩展 Tombola 类
class BingoCage(Tombola):

    def __init__(self, items):
        self._randomizer = random.SystemRandom()  # 生成随机字节序列
        self._items = []
        self.load(items)  # 委托 .load(...) 方法实现初始加载

    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)  # 使用 SystemRandom 实例的 .shuffle() 方法

    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):
        self.pick()

In [15]:
# 扩展 Tombola 类，实现抽象方法
class TumblingDrum(Tombola):

    def __init__(self, iterable):
        self._balls = []
        self.load(iterable)

    def load(self, iterable):
        self._balls.extend(iterable)
        shuffle(self._balls)

    def pick(self):
        return self._balls.pop()

In [16]:
# 覆盖了继承的 inspect 和 loaded 方法
class LotteryBlower(Tombola):

    def __init__(self, iterable):
        self._balls = list(iterable)  # 接受任何可迭代对象

    def load(self, iterable):
        self._balls.extend(iterable)

    def pick(self):
        try:
            position = random.randrange(len(self._balls))  # 如果范围为空，抛出 ValueError
        except ValueError:
            raise LookupError('pick from empty BingoCage')
        return self._balls.pop(position)  # 兼容 Tombola ，抛出 LookupError
    
    # 覆盖方法
    def loaded(self):
        return bool(self._balls)

    # 覆盖方法
    def inspect(self):
        return tuple(sorted(self._balls))

白鹅类型的一个基本特性：即使不继承，也有办法把一个类注册为抽象基类的虚拟子类。

注册虚拟子类的方式是在抽象基类上调用 register 方法。之后，注册的类会变成抽象基类的虚拟子类，而且 issubclass 和 isinstance 等函数都能识别，但是注册的类不会从抽象基类中继承任何方法或属性。

为了避免运行时错误，虚拟子类要实现所需的全部方法。

register 方法通常作为普通的函数调用，不过也可以作为装饰器使用。

In [17]:
# 注册为 Tombola 的虚拟子类
@Tombola.register
class TomboList(list):  # 扩展 list

    def pick(self):
        if self:  # 从 list 中继承 __bool__ 方法，列表不为空时返回 True
            position = random.randrange(len(self))
            return self.pop(position)  # 调用继承自 list 的 self.pop 方法
        else:
            raise LookupError('pop from empty TomboList')

    load = list.extend

    def loaded(self):
        return bool(self)  # 委托 bool 函数

    def inspect(self):
        return tuple(sorted(self))

In [18]:
issubclass(TomboList, Tombola)

True

In [19]:
t = TomboList(range(100))
isinstance(t, Tombola)

True

类的继承关系在特殊的类属性 \_\_mro__ 中指定，即方法解析顺序（Method Resolution Order），该属性按顺序列出类及其超类，Python 会按照这个顺序搜索方法

In [20]:
TomboList.__mro__

(__main__.TomboList, list, object)

使用到两个类属性，用它们内省类的继承关系
- \_\_subclass__()
    - 返回类的直接子类列表，不含虚拟子类
- \_abc_registry
    - 只有抽象基类有这个属性，值是一个 WeakSet 对象，即抽象类注册的虚拟子类的弱引用

In [21]:
TEST_FILE = 'tombola_tests.rst'
TEST_MSG = '{0:16} {1.attempted:2} tests, {1.failed:2} failed - {2}'


def test(cls, verbose=False):

    res = doctest.testfile(
            TEST_FILE,
            globs={'ConcreteTombola': cls},  # 要测试的类绑定到全局命名空间里的 ConcreteTombola 名称上
            verbose=verbose,
            optionflags=doctest.REPORT_ONLY_FIRST_FAILURE)
    tag = 'FAIL' if res.failed else 'OK'
    print(TEST_MSG.format(cls.__name__, res, tag))  # 输出测试结果

verbose = True
real_subclasses = Tombola.__subclasses__()  # 内存中存在的直接子代
virtual_subclasses = list(Tombola._abc_registry)  # Python 3.8 不支持

for cls in real_subclasses + virtual_subclasses:  # 找到各个子类，分别传给 test 函数
    test(cls, verbose)

AttributeError: type object 'Tombola' has no attribute '_abc_registry'

Struggle 是 abc.Sized 的子类，因为 abc.Sized 实现了一个特殊的类方法 \_\_subclasshook__
- 让抽象基类识别没有注册为子类的类
- 检查虚拟子类，返回 `True`

\_\_subclasshook__ 在白鹅类型中添加了一些鸭子类型的踪迹。

借助“白鹅类型”，可以使用抽象基类明确声明接口，而且类可以子类化抽象基类或使用抽象基类注册（无需再继承关系中确立静态的强链接），宣称它实现了某个接口。

日常使用中，我们与抽象基类的联系应该是创建现有抽象基类的子类，或者使用现有的抽象基类注册。此外，我们可能还会在 isinstance 检查中使用抽象基类。

In [22]:
class Struggle:
    def __len__(self):
        return 23

from collections import abc

isinstance(Struggle(), abc.Sized)

True

In [23]:
issubclass(Struggle, abc.Sized)

True

强类型和弱类型
- 如果一门语言很少隐式转换类型，说明它是强类型语言；如果经常这么做，说明它是弱类型语言
- 强类型语言：Java、C++、Python
- 弱类型语言：PHP、JavaScript、Perl

静态类型和动态类型
- 在编译时检查类型的语言是静态类型语言，在运行时检查类型的语言是动态类型语言
- 静态类型使得一些工具（编译器和 IDE）便于分析代码，找出错误和提供其他服务（优化、重构、等等）
- 动态类型便于代码重用，代码行数更少，而且能让接口自然或成为协议而不提早实行

Go 语言可以说是具有“静态鸭子类型”。与 Python 相比，对 Go 来说好像每个抽象基类都实现了 \_\_subclasshook__ 方法，它会检查函数的名称和签名，而我们自己从不需要继承或注册抽象基类。